In [174]:
import random
import scipy.io as sio
import scipy
import operator
import itertools
import collections
from tabulate import tabulate
import numpy as np

N         = 20  # Number of jobs in the queue
P         = 2*N # Number of processors,special case for infinite proc
kk        = 2   # Pairs of applications
qos       = 0.9 # QoS threshold

# loading data
def loaddata(k):
    mat_contents = scipy.io.loadmat('../tmp/dat-multi-'+str(k)+'.mat')
    oct_struct   = mat_contents['dat']
    X ,Y         = oct_struct['xx'], oct_struct['yy']
    X, Y         = X[0,0], Y[0,0]
    return X, Y
    
with open('../data/All_app.tsv') as f:
    app_name = f.read().split('\n')

X,Y       = loaddata(2)
n         = len(app_name)

In [175]:
job_slice = collections.namedtuple('job_slice',\
            ['job_id','name','app_id','type','work','qos','deadline'])

# generate a random job queue
class job_queue:
    def __init__(self, job_queue = []):
        self._job_queue = job_queue
    def random(self, N, qos, app_name):
        """
        :type N:          int: Number of jobs
        :type qos:        float: qos constraint
        :type app_name:   list[str]: application names
        :rtype: job_queue:list[job_slice]: Set of jobs
        """    
        # Harcoding qos job names
        app_name_qos   = ['x264','vips','swish','swaptions','streamcluster',\
                          'STREAM','fluidanimate','blackscholes']
        num_qos   = 0
        job_queue = []
        for i in range(N):
            j_work     = random.choice([100,500,1000])
            if num_qos < P/2:
                j_type     = random.choice(['qos', 'batch'])
            else:
                j_type     = 'batch'

            if j_type      == 'qos':
                  
                j_qos      = qos
                j_deadline = (1+(1-qos))*j_work
                #j_deadline = None
                num_qos    += 1
                indices    = [i for i,item in enumerate(app_name) \
                              if item in app_name_qos]
                j_iden     = random.choice(indices)
                j_name     = app_name[j_iden-1]
            elif j_type    == 'batch':
                j_deadline = random.choice(range(j_work,3000,100))
                j_qos      = None
                indices    = [i for i,item in enumerate(app_name) \
                              if item not in app_name_qos] 
                j_iden     = random.choice(indices)
                j_name     = app_name[j_iden-1]
            job = job_slice(i+10000, j_name, j_iden, j_type,  j_work, j_qos, j_deadline)
            job_queue.append(job);
        #job_queue.sort(key=operator.attrgetter('job'));
        self._job_queue = job_queue
     
    def __len__(self): 
        return len(self._job_queue)
    
    def __getitem__(self, position): 
        # TODO make it safer 
        if position < 9999:
            return self._job_queue[position]
        else:
            val = None
            for job in self._job_queue:
                if job.job_id == position:
                    val = job
            return val
    
    def append(self, item): 
        self._job_queue.append(item)
    
#     def remove(self, position): 
#         if position < 9999:
#             del self._job_queue[position]
#         else:
#             for i,job in enumerate(self._job_queue):
#                 if job.job_id == position:
#                     del self._job_queue[i]
    def remove(self, item): 
        #del self._job_queue[item]
        for i,job in enumerate(self._job_queue):
            if item is job:
                del self._job_queue[i]
    def sort(self,  *keyy ,reverse = False): 
        self._job_queue.sort(key=operator.attrgetter(*keyy), reverse = reverse )
    
    def __repr__(self):
        return tabulate(list([job for job in self._job_queue] ), headers = job_slice._fields)
    


def nchoosek_rep(array, k):
    return list(itertools.combinations_with_replacement(array, k))



In [176]:
jobs = job_queue()
jobs.random(N, qos, app_name)
jobs.sort('job_id')
print(jobs)

  job_id  name             app_id  type      work    qos    deadline
--------  -------------  --------  ------  ------  -----  ----------
   10000  dijkstra              8  qos       1000    0.9        1100
   10001  STREAM               21  qos        500    0.9         550
   10002  tsp                  26  qos        100    0.9         110
   10003  heartwall            10  batch      100                900
   10004  x264                  0  batch      500               1600
   10005  heartwall            10  batch     1000               2100
   10006  swish                25  batch      500               2000
   10007  blackscholes          4  batch     1000               2400
   10008  btree                 5  batch      500               2400
   10009  hotspot              12  batch      500               2600
   10010  backprop              2  batch      500               2300
   10011  dijkstra              8  qos        100    0.9         110
   10012  streamcluster        22 

In [181]:
#description

schedule = collections.namedtuple('schedule',['interval','jobs',\
                                              'speeds','est_speeds'])  
# scheduling jobs
class machine:
    def __init__(self, machine_id=0):
        self.utilization   = 0
        self.res           = 0
        self._performance  = []
        self._schedules    = [] # list of schedules
        self.machine_id    = machine_id
        ## TODO: variable leak
        self._job_queue    = job_queue([]) # jobs on the machine         
    def fit(self, Y, app_name):
        n         = len(app_name)
        # Harcoding qos job names
        app_name_qos   = ['x264','vips','swish','swaptions','streamcluster',\
                          'STREAM','fluidanimate','blackscholes']
        indices = [i for i,item in enumerate(app_name) \
                              if item in app_name_qos]

        b  = nchoosek_rep(range(n), 2)
        for idx,ii in enumerate(b):
            for i,j in zip(range(len(ii)),ii):
                if j in indices and Y[idx,i] < 0.9:
                    Y[idx,:] = 0
        self._performance = Y
    def update(self, job):
        
        self._job_queue.append(job)
        
        app_id   = list([job.app_id   for job in self._job_queue])
        W        = list([job.work     for job in self._job_queue])
        deadline = list([job.deadline for job in self._job_queue])
        
        res, A = self.__lin_prog(app_id, W, deadline, Y, n)   
        self.utilization = res.fun
        self._schedules  = (res, A)
        
        #TODO: add conditions related to deadline for removing a job
        if random.choice([0,1])==0:
            self._job_queue.remove(job)
            return False
        else:
            return True
    def __lin_prog(self, app_id, W, deadline, Y, n):        
        a  = nchoosek_rep(app_id, 2)
        aa = nchoosek_rep(range(len(app_id)), 2)
        b  = nchoosek_rep(range(n), 2)
        A  = np.zeros(shape=(len(app_id),len(a)))
        for i in range(len(a)):
            tmp = b.index(tuple(sorted(a[i])))
            sort_index = np.argsort(a[i])
            for j in sort_index:
                A[aa[i][j],i] = Y[tmp, j]

        A = np.concatenate([np.eye(len(app_id)), A.T])
        A = A.T
        res = scipy.optimize.linprog([1]*A.shape[1], A_eq=A, b_eq=W)
                                    #options={"disp": True})
        #np.set_printoptions(precision=2);print(res.fun)
        return (res, A)
class servers:
    def __init__(self):
        self.machines = []
        
    # This function adds a job to the current job_queue on machine
    # and returns true if the job could be successfully added
    
    # This function adds a new machine
#     def update(self, machine):
#         self.machines.append(machine)
#         self.machine_id.append(20000+len(self.machine_id))
    def create_machine(self):
        mac  = machine(20000+len(self.machines))
        return mac
    def __getitem__(self, position): 
        return self.machines[position]
    
    def __len__(self): 
        return len(self.machines)

In [182]:
serv = servers()
for job in jobs:
    val = False
    machines_checked = 0
    while val==False:
        if machines_checked < len(serv):
            mac = serv[machines_checked]
            machines_checked+=1
        else:
            mac  = serv.create_machine()
            mac.fit(Y,app_name)
            val = mac.update(job)
    jobs.remove(job)
    print('** MACHINE ID **', mac.machine_id )
    print(mac._job_queue )
    
    #np.set_printoptions(precision=2);print(res.fun)

# print(jobs)
# print(res.x.shape)
# print(A.shape)
# print(serv.machines[0]._job_queue)

** MACHINE ID ** 20000
  job_id  name         app_id  type      work  qos      deadline
--------  ---------  --------  ------  ------  -----  ----------
   10003  heartwall        10  batch      100                900
** MACHINE ID ** 20000
  job_id  name        app_id  type      work    qos    deadline
--------  --------  --------  ------  ------  -----  ----------
   10011  dijkstra         8  qos        100    0.9         110
** MACHINE ID ** 20000
  job_id  name      app_id  type      work  qos      deadline
--------  ------  --------  ------  ------  -----  ----------
   10019  DFS            7  batch      100                800


In [144]:
#sched_prelim = res.x
#app_id = sorted(list([job.app_id for job in mac._job_queue]))
#aa = app_id + nchoosek_rep(app_id, 2)

job_id = list([job.job_id for job in mac._job_queue])
aa = nchoosek_rep(job_id, 1) + nchoosek_rep(job_id, 2)

sched_prelim_pair = [pair for i,pair in enumerate(aa) if res.x[i] > 0.001]
sched_prelim_time = [res.x[i] for i,pair in enumerate(aa) if res.x[i] > 0.001]

print(sched_prelim_pair)
print(sched_prelim_time)

[(10000,), (10000, 10004), (10002, 10004)]
[802.00868223647024, 947.32591985134843, 249.17486942772356]


In [145]:
jobs.sort('deadline')
jobs.sort('type', reverse = True)
print(jobs)

#sched = []
#['interval','jobs',\
 #                                             'speeds','est_speeds'
# timee = 0
# for job in jobs: # to maintain importance order
#     scheduled_pairs = []
#     for pair in sched_prelim_pair:
#         if job.job_id in pair :
#              scheduled_pairs.append(schedule())

  job_id  name      app_id  type      work    qos    deadline
--------  ------  --------  ------  ------  -----  ----------
   10003  STREAM        21  qos        100    0.9         110
   10005  bfs            3  qos        500    0.9         550
   10004  nw            18  batch     1000               2200


In [146]:
list(zip( nchoosek_rep(range(n), 2), mac._performance))

[((0, 0), array([ 0.51,  0.51])),
 ((0, 1), array([ 0.76,  0.68])),
 ((0, 2), array([ 0.83,  0.61])),
 ((0, 3), array([ 0.,  0.])),
 ((0, 4), array([ 0.57,  0.66])),
 ((0, 5), array([ 0.49,  0.45])),
 ((0, 6), array([ 0.67,  0.77])),
 ((0, 7), array([ 0.86,  0.53])),
 ((0, 8), array([ 0.,  0.])),
 ((0, 9), array([ 0.76,  0.62])),
 ((0, 10), array([ 0.48,  0.49])),
 ((0, 11), array([ 0.94,  0.4 ])),
 ((0, 12), array([ 0.41,  0.23])),
 ((0, 13), array([ 0.73,  1.05])),
 ((0, 14), array([ 0.43,  0.72])),
 ((0, 15), array([ 0.49,  0.63])),
 ((0, 16), array([ 0.5 ,  0.08])),
 ((0, 17), array([ 0.54,  2.53])),
 ((0, 18), array([ 0.46,  0.21])),
 ((0, 19), array([ 0.63,  0.36])),
 ((0, 20), array([ 0.,  0.])),
 ((0, 21), array([ 0.22,  1.18])),
 ((0, 22), array([ 0.12,  0.56])),
 ((0, 23), array([ 0.,  0.])),
 ((0, 24), array([ 0.,  0.])),
 ((0, 25), array([ 0.1,  0.7])),
 ((0, 26), array([ 0.,  0.])),
 ((0, 27), array([ 0.28,  2.21])),
 ((1, 1), array([ 0.48,  0.48])),
 ((1, 2), array([ 0.87

In [169]:
print(server[0])

NameError: name 'server' is not defined

In [148]:
jobs.remove(10005)


In [149]:
print(jobs)

  job_id  name      app_id  type      work    qos    deadline
--------  ------  --------  ------  ------  -----  ----------
   10003  STREAM        21  qos        100    0.9         110
   10004  nw            18  batch     1000               2200
